$\newcommand{\To}{\Rightarrow}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import TFun, BoolType, NatType
from kernel import term
from kernel.term import Term, Var, Const, Lambda, Abs, Bound, Nat, Or, Eq, Forall, Exists, Implies, And
from data import nat
from logic import basic

basic.load_theory('nat')

## Lambda calculus

In the previous section, we discussed how to construct terms consisting of variables, constants, and function application. The relevant constructors are `Var`, `Const`, and `Comb`. In this section, we discuss construction of *lambda terms*, which completes the representation of terms in *lambda calculus*.

The motivation is as follows: we have already noted that terms can have function type. For example, in the previous section, we can declare a variable $f$ of type $nat \To nat$ by `Var("f", TFun(NatType, NatType))`. We have also encountered constants that have function type, for example the addition operator. However, we have not said anything about how to construct new examples of such functions.

In principle, any well-defined rule for computing the output from the input should be representable as a function. For example, there should be a function that takes as input a natural number $n$, and outputs $n+2$. In higher-order logic (also known as *simply-typed lambda calculus*), we can represent such functions as *lambda terms*. The above function can be written (in mathematical notation) as:

$$ \lambda n. n + 2 $$

Here $n$ (the variable right after $\lambda$) is known as a *bound variable*, in the sense that it is associated to the $\lambda$ sign directly in front of it, and is valid only in the scope of that $\lambda$ sign. It is important to note that *the name of the bound variable does not matter*. The expression $\lambda n. n + 2$ means the same thing as the expression $\lambda m. m + 2$. Both represent functions that add 2 to its input. We say that two terms are *$\alpha$-equivalent* if one can be changed to the other by changing the names of some bound variables.

We can construct a function term using `Lambda`.

In [3]:
n = Var('n', NatType)
f = Lambda(n, n + 2)
print(f)

%n::nat. n + 2


Note $\lambda$ is printed in ASCII using `%`. We can test that the name of bound variable does not matter by constructing `t` in another way:

In [4]:
m = Var('m', NatType)
f2 = Lambda(m, m + 2)
print(f2)
assert f == f2

%m::nat. m + 2


Functions taking several arguments can be constructed using multiple Lambdas. The following constructs a function that takes two natural numbers $x$ and $y$ as input, and returns $x + 2y$.

In [5]:
x = Var('x', NatType)
y = Var('y', NatType)
g = Lambda(x, Lambda(y, x + 2 * y))
print(g)

%x::nat. %y. x + 2 * y


This can be written more simply as follows:

In [6]:
# TODO: Lambda with multiple variables.

The types of $f$ and $g$ are as expected (recall `checked_get_type` will perform type-checking on the term, in addition to returning the type of the term).

In [7]:
print(f.checked_get_type())
print(g.checked_get_type())

nat => nat
nat => nat => nat


`Lambda` can also be used to construct predicates or binary relations.

In [8]:
P = Lambda(x, Or(Eq(x, 0), Eq(x, 2)))
print(P)

R = Lambda(x, Lambda(y, Eq(x, y + 2)))
print(R)

%x::nat. x = 0 | x = 2
%x::nat. %y. x = y + 2


## $\beta$-conversion

In the previous section, we constructed lambda terms using the `Lambda` constructor. These are supposed to represent functions. What happens when we apply such functions an argument? Well, initially nothing happens:

In [9]:
print(f)
t = f(Nat(3))
print(t)

%n::nat. n + 2
(%n::nat. n + 2) 3


The `Comb` constructor (invoked through the `__call__` method of $f$) simply combines its two arguments, performing no function evaluation. To actually evaluate a function application, we need to use the `beta_conv` method, so named because function evaluation in lambda calculus is called *$\beta$-conversion*.

In [10]:
t2 = t.beta_conv()
print(t2)

(3::nat) + 2


Now, the argument 2 is substituted into the function. More precisely, the function `beta_conv` assumes the input term is in the form `f x`, where `f` is a lambda term, and substitutes `x` for the bound variable of `f`. The addition $3+2$ is still not evaluated: the general rule is that no evaluation is performed unless explicitly called for. We will discuss evaluation of arithmetic on natural numbers in a later section.

Let's see a more complicated example:

Oops... Here `beta_conv` failed because the function part of $t_3$ is not a lambda term: it is a lambda term applied to 2. To fully evaluate $f_2$ on two arguments 2 and 3, we need to apply them one at a time, performing $\beta$-conversion:

In [11]:
print('g: ', g)
t3 = g(Nat(3), Nat(4))
print('t3:', t3)

t4 = t3.beta_conv()   # raises TermException

g:  %x::nat. %y. x + 2 * y
t3: (%x::nat. %y. x + 2 * y) 3 4


TermException: beta_conv: input is not in the form (%x. t1) t2.

In [12]:
t3 = g(Nat(3)).beta_conv()
print('t3:', t3)
t4 = t3(Nat(4)).beta_conv()
print('t4:', t4)

t3: %y::nat. 3 + 2 * y
t4: (3::nat) + 2 * 4


A more convenient method is `beta_norm`, which performs all $\beta$-conversions on subterms:

In [13]:
t5 = g(Nat(3),Nat(4)).beta_norm()
print(t5)

(3::nat) + 2 * 4


## Quantifiers in predicate logic

Predicate logic extends propositional logic by adding two quantifiers: forall ($\forall$) and exists ($\exists$). In higher-order logic, both operators are represented as constants of type $('a \To bool) \To bool$. This can be explained as follows, taking the forall quantifier as an example. A forall expression in mathematics has the form

$$ \forall x. P(x) $$

Here $x$ is a bound variable. In (untyped) first-order logic, there are only two types of terms: objects and propositions, and $x$ can only range over objects. The main distinction between higher-order and first-order logic is that in higher-order logic, the bound variable of quantifiers can be of any type, including function types. Hence, we designate the type of the bound variable by the type variable $'a$. Then, the predicate $P$ has type $'a \To bool$. Any forall expression is a function taking a predicate $P$ of type $'a \To bool$ as input, and outputs a boolean value (whether $P$ is true on all of $'a$). Hence, its type must be $('a \To bool) \To bool$.

Forall and exists expressions are constructed as follows.

In [14]:
x = Var("x", NatType)
t1 = Forall(x, Implies(x > 2, x > 1))
print('t1:', t1)
t2 = Exists(x, And(x > 2, x < 4))
print('t2:', t2)

t1: !x::nat. x > 2 --> x > 1
t2: ?x::nat. x > 2 & x < 4


The type of $t_1$ and $t_2$ are booleans, as expected.

In [15]:
print(t1.checked_get_type())
print(t2.checked_get_type())

bool
bool


## de Bruijn indices

When representing terms in higher-order logic, we would like to be able to quickly tell whether two terms are $\alpha$-equivalent. This motivates the use of *de Bruijn index* (named after Dutch mathematician Nicolaas Govert de Bruijn). Following this method, the bound variables are (in principle) unnamed, and whenever one needs to refer to a bound variable, one uses a sign $B_i$ where $i$ counts the depth of the location of reference with respect to the lambda sign of that variable. We follow the convention that the counting begins at 0. For example, the above function is represented using de Bruijn index as:

$$ \lambda\_. B_0 + 2 $$

Here we use an underscore to denote a bound variable that is unnamed. Another example: the expression $\lambda x. \lambda y. x + y$ is represented as $\lambda\_. \lambda\_. B_1 + B_0$ using de Bruijn indices. This is because the location where $x$ occurs is separated from the $\lambda$ sign that bounds it (the first $\lambda$ sign) by one $\lambda$ sign in the middle, while the location where $y$ occurs is directly after the $\lambda$ sign that bounds it (the second $\lambda$ sign).

The use of de Bruijn indices is revealed by looking at the `repr` of a lambda term:

In [16]:
x = Var('x', NatType)
t = Lambda(x, x + 1)
print(repr(t))

Abs(x, nat, Comb(Comb(Const(plus, nat => nat => nat), Bound(0)), Const(one, nat)))


Here, `Abs` is the constructor for a lambda term. The first argument is the *suggested* name of the bound variable. It is used for printing only (and perhaps as a starting point when names of new variables need to be invented during proof). The second argument is the type of the bound variable, which *is* significant (different types of bound variables give different terms). The third argument is the body of the lambda term. In the body, bound variables are refered to by `Bound(n)`, where $n$ is a natural number.

Let us examine a more complex lambda expression:

In [17]:
x = Var('x', NatType)
y = Var('y', NatType)
t = Lambda(x, Lambda(y, x + y))
print(t)
print(repr(t))

%x::nat. %y. x + y
Abs(x, nat, Abs(y, nat, Comb(Comb(Const(plus, nat => nat => nat), Bound(1)), Bound(0))))


While we are at it, let us also examine the representation of forall and exists terms:

In [18]:
print(repr(Forall(x, x >= 0)))

Comb(Const(all, (nat => bool) => bool), Abs(x, nat, Comb(Comb(Const(greater_eq, nat => nat => bool), Bound(0)), Const(zero, nat))))


In [19]:
print(repr(Exists(x, x < 1)))

Comb(Const(exists, (nat => bool) => bool), Abs(x, nat, Comb(Comb(Const(less, nat => nat => bool), Bound(0)), Const(one, nat))))


After understanding the de Bruijn representation, we can also creater lambda terms directly using the `Abs` and `Bound` constructors. This is seldomly necessary, but we show it here to illustrate the concepts:

In [20]:
t = Abs('x', NatType, nat.plus(Bound(0), nat.one))
print(t)
assert t == Lambda(x, x + 1)

%x::nat. x + 1
